## Lists and Dictionaries

### Slice


- 实现了__getitem__和__setitem__方法的class的对象，都可以使用slice

- slice允许越界的index访问

- slice assignment没有长度相等的要求

In [1]:
a = ['a', 'b', 'c']
a[1:2] = [2, 3, 4] # length of slice assignments don't need to be the same
print(a)

['a', 2, 3, 4, 'c']


- stride和slice

stride： somelist[start:end:stride]

使用stride时不建议specify start和end, 可读性较差

如有需要，用一步slice单独实现。

In [2]:
temp = a[1:3] # instead of a[1:3:-1]
b = temp[::-1] # 用stride逆序的方法
print(b)

[3, 2]


- slice会发生shallow copy, 修改slicing的结果不会修改原list; 

  itertools的islice方法可以避免copy，用法与slice类似.
  
  islice(iterable, [start, ] stop [, step])
  
  只允许None或正数值。

In [3]:
import itertools

values = list(range(10))
first_five = list(itertools.islice(values, 5))
odds = list(itertools.islice(values, 1, None,2))
evens = list(itertools.islice(values, None, None, 2))
print('first five: ', first_five)
print('odds: ', odds)
print('evens: ', evens)

first five:  [0, 1, 2, 3, 4]
odds:  [1, 3, 5, 7, 9]
evens:  [0, 2, 4, 6, 8]


### Sort by Complex Criteria Using the `key` parameter

借助sort的key参数，灵活的使用sort

In [4]:
class Tool:
    def __init__(self, name, weight):
        self.name = name
        self.weight = weight
    
    def __repr__(self):
        return f'Tool({self.name}, {self.weight})'

tools = [
    Tool('hammer', 1.25), 
    Tool('screwdriver', 0.5),
    Tool('chisel', 0.25)    
]

tools.sort(key=lambda x: (x.name, x.weight), reverse=True) # do transformations on the values
'''
tuple自带了比较机制, 从左至右，同一位置元素相同则继续，不同就得到比较结果。
可以借助tuple实现multiple sorting criteria
'''
print(tools)

[Tool(screwdriver, 0.5), Tool(hammer, 1.25), Tool(chisel, 0.25)]


### Be Cautious When Relying on `dict` Insertion Ordering

python 3.5及之前的版本，dict的遍历顺序是随机的，与插入顺序无关。

但3.7开始, "dict will preserve insertion order"

这意味着`keys`, `values`, `items`, `popitem`都会保序.

但派生collections.abs的MutableMapping得到的类，并不具有这个性质。

如果要依赖dict的保序性，应该check变量类型

In [5]:
class Student:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
a = Student('Bob', 12)
for key, value in a.__dict__.items(): # 按照声明顺序
    print(f'{key} = {value}')

name = Bob
age = 12


### Prefer `get` Over `in` and `KeyError` to Handle Missing Dictionary Keys

处理missing keys的方法有四种

- 用in判断
- catch KeyError
- get(key, value)
> value[optional]:
A value to return if the specified key does not exist. Default value None
 
  会先判断key, 再创建value. 当value的construct has a high cost的时候，推荐使用get

- setdefault
可以直接添加missing key到dict中。
但这时可以使用`defaultdict`

In [7]:
if temp := counters.get('c') is None:
        temp = 'no'
'''
if 'c' in counters:
    temp = counters['c']
else:
    temp = 'no'
'''
print(temp)


no


In [8]:
'''
也可以使用setdefault, 赋值的同时，在容器中初始化新的key.(直接引用value，不是拷贝)
- 可读性较差
- not efficient 
  default value(比如是一个set())每次都会构建，无论key是否在dict中.
'''
counters.setdefault('c', []).append("value")
print(counters)

{'c': ['value']}


### Prefer `defaultdict` Over `setdefault` to Handle Missing Items in Internal State

创建时给定一个返回默认值的function

注意
- defaultdict不再保序
- defaultdict构造默认值的function不能处理有参的情况
 > The function passed to defaultdict must not require any arguments,which makes it impossible to have the default value depend on the key being accessed

In [12]:
from collections import defaultdict

a = defaultdict(set)
b = defaultdict(lambda : "None") # 匿名lambda

print(a[1])
print(b[0])

set()
None


### Know How to Construct Key-Dependent Default Values with __missing__

defaultdict的默认值不能依赖于key, 或者说提供默认值的函数不能是有参的。

为此可以重载`__missing__`方法

In [17]:
class Pictures(dict):
    def __missing__(self, key):
        value = 'value: ' + key # key-dependent default value
        self[key] = value # 增加missing key
        return value
    
path = 'pic1.jpg'
pictures = Pictures()
value = pictures[path]
print(value)

value: pic1.jpg
